## Data Scraping - entradas.com

Como primera fuente he decididio escoger la web de 'entradas.com', seleccionando específicamente conciertos y festivales que estos tienen disponibles. 

En cuanto al proceso de scrapeo que se lleva a cabo, he decicido centrarme en los siguientes datos: 
- Artista.
- Número de eventos que tienen disponibles.
- Fechas, en este caso algunos eventos presentan dos fechas o fechas en ese rango, por lo que muestro ambas fechas ('fecha1' y 'fecha2')
- Precio de salida de las entradas al evento. 
- Enlace al evento. 

Para comenzar con la búsqueda, decido seguir el método selenium de scrapeo. 

Para ello, importo las librerías que necesito, y aquellas funciones que pueden ser útiles en mi búsqueda. 

In [1]:
import pandas as pd
import numpy as np
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver # driver de selenium
import time   # para manejar tiempo

In [2]:
from selenium.webdriver.chrome.options import Options   # opciones del driver
from selenium.webdriver.common.by import By   # para buscar segun tag, id, class, etc...
from selenium.webdriver.support.ui import WebDriverWait   # es para esperar
from selenium.webdriver.support import expected_conditions as EC  # condiciones esperadas...
from selenium.webdriver import ActionChains as AC   # acciones encadenadas, rollo doble click
from selenium.webdriver.common.keys import Keys  # manejar teclas

In [3]:
# Además, hago la 'driver configuration'

opciones = Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
opciones.add_argument('--incognito')

#### 1. Conecto con la página WEB

In [4]:
PATH = 'driver/chromedriver'

driver = webdriver.Chrome()   # abre una ventana de chrome 
                                # una vez que he abierto una ventana, puedo trabajar siempre sobre esta misma.

In [5]:
# Tenemos 5 páginas en total, por lo que a partir de aquí iremos llamando a la url para que cambie de página. 

url = 'https://www.entradas.com/city/madrid-370/conciertos-y-festivales-85/'

driver.get(url)

- Conecto con la Web de entradas.com para trabajar sobre esta, y hacer scraping de los datos que quiero. Utilizaré el método de Scrapeo de selenium.

#### 2. Aceptar las cookies 

- Lo primero que hago es darle a Aceptas las cookies del sitio web. 
- Para ello tengo un problema inicial, ya que está categorizada como una pestaña "shadow" por lo que cuando busco la opción de hacer click al botón de aceptar, aparece como una pestaña / lista vacía. 

In [6]:
div_shadow = driver.find_element(By.XPATH, '//div[@class="cmpwrapper"]')

In [7]:
# Check if the element has a Shadow DOM using JavaScript
has_shadow_dom = driver.execute_script('return arguments[0].shadowRoot !== null', div_shadow)

if has_shadow_dom:
    print("Element has a Shadow DOM")
else:
    print("Element does not have a Shadow DOM")

Element has a Shadow DOM


In [8]:
shadow_root = driver.execute_script('return arguments[0].shadowRoot', div_shadow)

In [9]:
shadow_root.find_elements(By.CSS_SELECTOR, '#cmpwelcomebtnyes > a')[0].click()

- Una vez consigo traspasar la barrera del 'Shadow DOM', entonces puedo comenzar con el scrapeo de las tablas que necesito para mi estudio. 

#### 3. Empezamos el scrapeo por eventos en tabla

In [10]:
tabla = '//product-group-item'

In [11]:
table = driver.find_elements(By.XPATH, tabla)
table

[<selenium.webdriver.remote.webelement.WebElement (session="fd0d5c95b8fc3784ead1aa0fe516404a", element="C500D52C27E9E74D7A3FD44AB714F488_element_42")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fd0d5c95b8fc3784ead1aa0fe516404a", element="C500D52C27E9E74D7A3FD44AB714F488_element_43")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fd0d5c95b8fc3784ead1aa0fe516404a", element="C500D52C27E9E74D7A3FD44AB714F488_element_44")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fd0d5c95b8fc3784ead1aa0fe516404a", element="C500D52C27E9E74D7A3FD44AB714F488_element_45")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fd0d5c95b8fc3784ead1aa0fe516404a", element="C500D52C27E9E74D7A3FD44AB714F488_element_46")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fd0d5c95b8fc3784ead1aa0fe516404a", element="C500D52C27E9E74D7A3FD44AB714F488_element_47")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fd0d5c95b8fc3784ead1aa0fe516

In [12]:
type(table)

list

In [13]:
elementos = driver.find_elements(By.XPATH, '//*[@id="list-item-0"]')
elementos

[<selenium.webdriver.remote.webelement.WebElement (session="fd0d5c95b8fc3784ead1aa0fe516404a", element="C500D52C27E9E74D7A3FD44AB714F488_element_62")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fd0d5c95b8fc3784ead1aa0fe516404a", element="C500D52C27E9E74D7A3FD44AB714F488_element_63")>]

In [14]:
type(elementos)

list

In [15]:
elementos = elementos[0].text.split('\n')
elementos

['Cariño',
 'Inverfest 2024',
 '13/01/20241 Evento',
 ' desde 19,80 €',
 'Mostrar 1 Evento']

In [16]:
elements = driver.find_elements(By.XPATH, '//div[@class="search-result-content"]')
elements

[<selenium.webdriver.remote.webelement.WebElement (session="fd0d5c95b8fc3784ead1aa0fe516404a", element="C500D52C27E9E74D7A3FD44AB714F488_element_64")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fd0d5c95b8fc3784ead1aa0fe516404a", element="C500D52C27E9E74D7A3FD44AB714F488_element_36")>]

In [17]:
data = []

for element in elements:
    artist_name = element.find_elements(By.XPATH, '//div[@class="event-listing-city theme-text-color"]')
    astists = [artist.text for artist in artist_name]
    date_elements = element.find_elements(By.XPATH, '//div//span[@class="listing-data theme-text-color"]//span')
    dates = [date.text for date in date_elements]
    price = element.find_elements(By.XPATH, '//div/listing-cta/event-status/div/span/span')
    precios = [precio.text for precio in price]
    enlace = element.find_elements(By.XPATH, '//*[@class = "btn theme-button-radius btn-sm btn-block btn-primary theme-interaction-btn-bg theme-btn-font-color theme-btn-font-color-hover"]')
    enlaces = [enl.get_attribute('href') for enl in enlace]

    data.append([astists, dates, precios, enlaces])

for entry in data:
    print(entry)

[['Cariño', 'Iván Ferreiro', 'The World Of Hans Zimmer', 'Triángulo de Amor Bizarro', 'Nico B', 'Cala Vento', 'Sidonie', 'Belako - Sigo regando', 'Biznaga', 'Morreo - Alegría Club!', 'Diego el Cigala - Obras Maestras', 'Symphonic Rhapsody of Queen', 'Sexy Zebras', 'Biografía - Rafa Sánchez, de la unión', 'OBK', 'El Consorcio - Toda una vida', 'Rock en Familia - Hijos del Rock and Roll', 'Rock en Familia - I Love Rock and Roll', 'Rock en Familia - Queenmanía', 'Rock en Familia - I Love Heavy Metal'], ['13/01/2024', '1 Evento', '13/01/2024', '1 Evento', '24/04/2024 ─ 29/04/2024', '2 eventos', '29/12/2023', '1 Evento', '14/01/2024', '1 Evento', '01/12/2023', '1 Evento', '12/01/2024 ─ 14/01/2024', '2 eventos', '20/01/2024', '1 Evento', '26/01/2024', '1 Evento', '18/01/2024', '1 Evento', '11/12/2023 ─ 18/12/2023', '2 eventos', '16/01/2024', '1 Evento', '01/02/2024', '1 Evento', '11/12/2023', '1 Evento', '13/01/2024 ─ 27/01/2024', '3 eventos', '27/11/2023 ─ 28/11/2023', '2 eventos', '05/11/2

In [18]:
artistas = entry[0]
fecha_eventos = entry[1]
precios = entry[2]

In [19]:
fechas = []
num_eventos = []

for i in range(0, len(fecha_eventos), 2):
    fechas.append(fecha_eventos[i])

for i in range(1, len(fecha_eventos), 2):
    num_eventos.append(fecha_eventos[i])

print(fechas)
print(num_eventos)

['13/01/2024', '13/01/2024', '24/04/2024 ─ 29/04/2024', '29/12/2023', '14/01/2024', '01/12/2023', '12/01/2024 ─ 14/01/2024', '20/01/2024', '26/01/2024', '18/01/2024', '11/12/2023 ─ 18/12/2023', '16/01/2024', '01/02/2024', '11/12/2023', '13/01/2024 ─ 27/01/2024', '27/11/2023 ─ 28/11/2023', '05/11/2023 ─ 10/03/2024', '03/12/2023', '17/12/2023 ─ 14/04/2024', '19/11/2023 ─ 24/03/2024']
['1 Evento', '1 Evento', '2 eventos', '1 Evento', '1 Evento', '1 Evento', '2 eventos', '1 Evento', '1 Evento', '1 Evento', '2 eventos', '1 Evento', '1 Evento', '1 Evento', '3 eventos', '2 eventos', '2 eventos', '1 Evento', '2 eventos', '2 eventos']


In [20]:
import pandas as pd

In [21]:
datos_1 = pd.DataFrame({'artista': artistas, 'fechas': fechas, 'eventos': num_eventos, 'precios': precios, 'enlaces': enlaces})
datos_1

,artista,fechas,eventos,precios,enlaces
0,Cariño,13/01/2024,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/carino/carino-...
1,Iván Ferreiro,13/01/2024,1 Evento,"desde 38,50 €",https://www.entradas.com/artist/ivan-ferreiro/
2,The World Of Hans Zimmer,24/04/2024 ─ 29/04/2024,2 eventos,"desde 55,00 €",https://www.entradas.com/artist/hans-zimmer/
3,Triángulo de Amor Bizarro,29/12/2023,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/triangulo-de-a...
4,Nico B,14/01/2024,1 Evento,"desde 13,50 €",https://www.entradas.com/artist/nico-b/
5,Cala Vento,01/12/2023,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/cala-vento/
6,Sidonie,12/01/2024 ─ 14/01/2024,2 eventos,"desde 27,50 €",https://www.entradas.com/artist/sidonie/sidoni...
7,Belako - Sigo regando,20/01/2024,1 Evento,"desde 17,60 €",https://www.entradas.com/artist/belako/
8,Biznaga,26/01/2024,1 Evento,"desde 18,70 €",https://www.entradas.com/artist/biznaga/
9,Morreo - Alegría Club!,18/01/2024,1 Evento,"desde 14,50 €",https://www.entradas.com/artist/morreo/


#### Para entrar en la siguiente página (Pág. 2)

In [22]:
driver.find_elements(By.XPATH, '//div/product-listing/listing-pagination/nav/ul/pagination-item[6]/a/i')[0].click()

In [23]:
table = driver.find_elements(By.XPATH, '//product-group-item')

In [24]:
elements = driver.find_elements(By.XPATH, '//div[@class="search-result-content"]')
elements

[<selenium.webdriver.remote.webelement.WebElement (session="fd0d5c95b8fc3784ead1aa0fe516404a", element="C500D52C27E9E74D7A3FD44AB714F488_element_64")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fd0d5c95b8fc3784ead1aa0fe516404a", element="C500D52C27E9E74D7A3FD44AB714F488_element_233")>]

In [25]:
data = []

for element in elements:
    artist_name = element.find_elements(By.XPATH, '//div[@class="event-listing-city theme-text-color"]')
    astists = [artist.text for artist in artist_name]
    date_elements = element.find_elements(By.XPATH, '//div//span[@class="listing-data theme-text-color"]//span')
    dates = [date.text for date in date_elements]
    price = element.find_elements(By.XPATH, '//div/listing-cta/event-status/div/span/span')
    precios = [precio.text for precio in price]
    enlace = element.find_elements(By.XPATH, '//div[2]/product-group-item/listing-item/article/div/listing-cta/a')
    enlaces = [enl.get_attribute('href') for enl in enlace]
    
    data.append([astists, dates, precios, enlaces])

for entry in data:
    print(entry)

[['Morgan', 'Los Zigarros', 'Queralt Lahoz', 'Noa', 'Mocedades - Avanzando hacia el futuro', 'John Grant', 'Callas en concierto - En holograma', 'Zenet', 'Laura Pausini', 'Concierto Sinfónico de Abono B18 (Jueves)', 'Concierto Sinfónico de Abono B18 (Viernes)', 'Concierto Sinfónico de Abono B16 (Viernes)', 'Concierto Sinfónico de Abono B16 (Jueves)', 'Concierto Sinfónico de Abono A13 (Jueves)', 'Concierto Sinfónico de Abono A15 (Jueves)', 'Concierto Sinfónico de Abono A15 (Viernes)', 'Concierto Sinfónico de Abono B12 (Viernes)', 'Concierto Sinfónico de Abono A19 (Viernes)', 'Concierto Sinfónico de Abono A7 (Viernes)', 'Concierto Sinfónico de Abono B10 (Viernes)'], ['19/01/2024', '1 Evento', '11/01/2024 ─ 12/01/2024', '2 eventos', '01/12/2023 ─ 02/12/2023', '2 eventos', '19/03/2024', '1 Evento', '08/01/2024', '1 Evento', '17/11/2023', '1 Evento', '11/11/2023 ─ 09/12/2023', '2 eventos', '26/11/2023', '1 Evento', '27/01/2024', '1 Evento', '18/04/2024', '1 Evento', '19/04/2024', '1 Evento'

In [26]:
artistas = entry[0]
fecha_eventos = entry[1]
precios = entry[2]

In [27]:
fechas = []
num_eventos = []

for i in range(0, len(fecha_eventos), 2):
    fechas.append(fecha_eventos[i])

for i in range(1, len(fecha_eventos), 2):
    num_eventos.append(fecha_eventos[i])

print(fechas)
print(num_eventos)

['19/01/2024', '11/01/2024 ─ 12/01/2024', '01/12/2023 ─ 02/12/2023', '19/03/2024', '08/01/2024', '17/11/2023', '11/11/2023 ─ 09/12/2023', '26/11/2023', '27/01/2024', '18/04/2024', '19/04/2024', '22/03/2024', '21/03/2024', '22/02/2024', '14/03/2024', '15/03/2024', '16/02/2024', '26/04/2024', '24/11/2023', '26/01/2024']
['1 Evento', '2 eventos', '2 eventos', '1 Evento', '1 Evento', '1 Evento', '2 eventos', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento']


In [28]:
datos_2 = pd.DataFrame({'artista': artistas, 'fechas': fechas, 'eventos': num_eventos, 'precios': precios, 'enlaces': enlaces})

datos_2

,artista,fechas,eventos,precios,enlaces
0,Morgan,19/01/2024,1 Evento,"desde 29,70 €",https://www.entradas.com/artist/morgan/
1,Los Zigarros,11/01/2024 ─ 12/01/2024,2 eventos,"desde 30,80 €",https://www.entradas.com/artist/los-zigarros/
2,Queralt Lahoz,01/12/2023 ─ 02/12/2023,2 eventos,"desde 23,00 €",https://www.entradas.com/artist/queralt-lahoz/
3,Noa,19/03/2024,1 Evento,"desde 30,00 €",https://www.entradas.com/artist/noa/
4,Mocedades - Avanzando hacia el futuro,08/01/2024,1 Evento,"desde 30,00 €",https://www.entradas.com/artist/mocedades/moce...
5,John Grant,17/11/2023,1 Evento,"desde 20,00 €",https://www.entradas.com/artist/john-grant/
6,Callas en concierto - En holograma,11/11/2023 ─ 09/12/2023,2 eventos,"desde 18,00 €",https://www.entradas.com/artist/maria-callas/
7,Zenet,26/11/2023,1 Evento,"desde 18,00 €",https://www.entradas.com/artist/zenet/
8,Laura Pausini,27/01/2024,1 Evento,"desde 30,00 €",https://www.entradas.com/artist/laura-pausini/
9,Concierto Sinfónico de Abono B18 (Jueves),18/04/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...


#### Para entrar en la siguiente página (Pág. 3)

In [29]:
driver.find_elements(By.XPATH, '//div/product-listing/listing-pagination/nav/ul/pagination-item[7]/a/i')[0].click()

In [30]:
table = driver.find_elements(By.XPATH, '//product-group-item')

In [31]:
elements = driver.find_elements(By.XPATH, '//div[@class="search-result-content"]')
elements

[<selenium.webdriver.remote.webelement.WebElement (session="fd0d5c95b8fc3784ead1aa0fe516404a", element="C500D52C27E9E74D7A3FD44AB714F488_element_64")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fd0d5c95b8fc3784ead1aa0fe516404a", element="C500D52C27E9E74D7A3FD44AB714F488_element_374")>]

In [32]:
data = []

for element in elements:
    artist_name = element.find_elements(By.XPATH, '//div[@class="event-listing-city theme-text-color"]')
    astists = [artist.text for artist in artist_name]
    date_elements = element.find_elements(By.XPATH, '//div//span[@class="listing-data theme-text-color"]//span')
    dates = [date.text for date in date_elements]
    price = element.find_elements(By.XPATH, '//div/listing-cta/event-status/div/span/span')
    precios = [precio.text for precio in price]
    enlace = element.find_elements(By.XPATH, '//div[2]/product-group-item/listing-item/article/div/listing-cta/a')
    enlaces = [enl.get_attribute('href') for enl in enlace]
    
    data.append([astists, dates, precios, enlaces])

for entry in data:
    print(entry)

[['Concierto Sinfónico de Abono B14 (Jueves)', 'Concierto Sinfónico de Abono B10 (Viernes)', 'Concierto Sinfónico de Abono A11 (Viernes)', 'Concierto Sinfónico de Abono B6 (Viernes)', 'Concierto Sinfónico de Abono A9 (Jueves)', 'Concierto Sinfónico de Abono B8 (Jueves)', 'Concierto Sinfónico de Abono B8 (Viernes)', 'Concierto Sinfónico de Abono A17 (Viernes)', 'Concierto Sinfónico de Abono A7 (Jueves)', 'CONCIERTO DE GALARDONADOS EL CONCURSO JUVENTUDES MUSICALES', 'JÓVENES MÚSICOS I', 'JÓVENES MÚSICOS III', 'Ensayos Ciclo Sinfónico', 'Concierto Sinfónico de Abono A17 (Jueves)', 'Concierto Sinfónico de Abono A11 (Jueves)', 'Concierto Sinfónico de Abono B20 (Viernes)', 'Concierto Sinfónico de Abono A19 (Jueves)', 'Concierto Sinfónico de Abono B10 (Jueves)', 'Concierto Sinfónico de Abono B14 (Viernes)', 'JÓVENES MÚSICOS II'], ['29/02/2024', '1 Evento', '26/01/2024', '1 Evento', '09/02/2024', '1 Evento', '10/11/2023', '1 Evento', '18/01/2024', '1 Evento', '11/01/2024', '1 Evento', '12/01/2

In [33]:
artistas = entry[0]
fecha_eventos = entry[1]
precios = entry[2]

In [34]:
fechas = []
num_eventos = []

for i in range(0, len(fecha_eventos), 2):
    fechas.append(fecha_eventos[i])

for i in range(1, len(fecha_eventos), 2):
    num_eventos.append(fecha_eventos[i])

print(fechas)
print(num_eventos)

['29/02/2024', '26/01/2024', '09/02/2024', '10/11/2023', '18/01/2024', '11/01/2024', '12/01/2024', '12/04/2024', '23/11/2023', '17/05/2024', '17/11/2023', '05/04/2024', '10/11/2023 ─ 09/05/2024', '11/04/2024', '08/02/2024', '10/05/2024', '25/04/2024', '25/01/2024', '01/03/2024', '02/02/2024']
['1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '15 eventos', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento']


In [35]:
datos_3 = pd.DataFrame({'artista': artistas, 'fechas': fechas, 'eventos': num_eventos, 'precios': precios, 'enlaces': enlaces})

datos_3

,artista,fechas,eventos,precios,enlaces
0,Concierto Sinfónico de Abono B14 (Jueves),29/02/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
1,Concierto Sinfónico de Abono B10 (Viernes),26/01/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
2,Concierto Sinfónico de Abono A11 (Viernes),09/02/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
3,Concierto Sinfónico de Abono B6 (Viernes),10/11/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
4,Concierto Sinfónico de Abono A9 (Jueves),18/01/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
5,Concierto Sinfónico de Abono B8 (Jueves),11/01/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
6,Concierto Sinfónico de Abono B8 (Viernes),12/01/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
7,Concierto Sinfónico de Abono A17 (Viernes),12/04/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
8,Concierto Sinfónico de Abono A7 (Jueves),23/11/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
9,CONCIERTO DE GALARDONADOS EL CONCURSO JUVENTUD...,17/05/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...


#### Para entrar en la siguiente página (Pág. 4)

In [36]:
driver.find_elements(By.XPATH, '//div/product-listing/listing-pagination/nav/ul/pagination-item[7]/a/i')[0].click()

In [37]:
table = driver.find_elements(By.XPATH, '//product-group-item')

In [38]:
elements = driver.find_elements(By.XPATH, '//div[@class="search-result-content"]')
elements

[<selenium.webdriver.remote.webelement.WebElement (session="fd0d5c95b8fc3784ead1aa0fe516404a", element="C500D52C27E9E74D7A3FD44AB714F488_element_64")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fd0d5c95b8fc3784ead1aa0fe516404a", element="C500D52C27E9E74D7A3FD44AB714F488_element_480")>]

In [39]:
data = []

for element in elements:
    artist_name = element.find_elements(By.XPATH, '//div[@class="event-listing-city theme-text-color"]')
    astists = [artist.text for artist in artist_name]
    date_elements = element.find_elements(By.XPATH, '//div//span[@class="listing-data theme-text-color"]//span')
    dates = [date.text for date in date_elements]
    price = element.find_elements(By.XPATH, '//div/listing-cta/event-status/div/span/span')
    precios = [precio.text for precio in price]
    enlace = element.find_elements(By.XPATH, '//div[2]/product-group-item/listing-item/article/div/listing-cta/a')
    enlaces = [enl.get_attribute('href') for enl in enlace]
    
    data.append([astists, dates, precios, enlaces])

for entry in data:
    print(entry)

[['JÓVENES MÚSICOS III', 'Concierto Sinfónico de Abono A11 (Jueves)', 'Concierto Sinfónico de Abono A17 (Jueves)', 'Maximiliano Calvo - El Gallo', 'Ricardo Moya', 'Bestia Bebé', 'Paul Alone', 'McEnroe', 'Grande Amore', 'Delgao', 'Christina Rosenvinge', 'Rubén Pozo', 'Ariel Rot - 25 años hablando solo', 'Israel B - EED1HS', 'Austin TV', 'Francisco', 'Raúl Paz', 'Mark Guiliana', 'Billy Cobham Spectrum 50 Project', 'Antonio Lizana', 'Shai Maestro', 'Antonio Rey y amigos', 'Theo Croker Quartet', 'Stick Men & Gilipojazz', 'Scary Pockets', 'The Cookers', 'Richard Bona', 'Takuya Kuroda', 'Matthew Halsall', 'Fatoumata Diawara', 'Calibro 35', 'Martirio & Chano Domínguez', 'Kassa Overall', 'Moisés P. Sánchez', 'El Kanka: Ruta Firestone 2023', 'Nando Agüeros', 'Jason Derulo', 'Zree and more', '“Skybridges”', 'Tina Turner Sinfónico', 'Elvis in my life', 'Concierto Conmemorativo 50 Aniversario', 'Lo sugerente', 'Walking Jazz "AtriLogy"', 'Lara Bello flamenco', '“Dedication II”', 'Bloomsound', 'Orqu

In [40]:
artistas = entry[0]
fecha_eventos = entry[1]
precios = entry[2]

In [41]:
fechas = []
num_eventos = []

for i in range(0, len(fecha_eventos), 2):
    fechas.append(fecha_eventos[i])

for i in range(1, len(fecha_eventos), 2):
    num_eventos.append(fecha_eventos[i])

print(fechas)
print(num_eventos)

['05/04/2024', '08/02/2024', '11/04/2024', '11/01/2024', '21/01/2024', '28/01/2024', '01/02/2024', '13/01/2024', '20/01/2024', '27/01/2024', '10/01/2024', '25/01/2024', '25/01/2024', '25/01/2024', '31/01/2024', '12/01/2024 ─ 23/03/2024', '07/11/2023', '20/11/2023', '17/11/2023', '30/11/2023', '12/11/2023', '05/11/2023', '09/11/2023', '05/11/2023', '09/02/2024', '09/11/2023', '10/11/2023', '07/11/2023', '19/11/2023', '21/11/2023', '24/11/2023', '14/11/2023', '25/11/2023', '18/11/2023', '23/11/2023', '28/01/2024', '06/12/2023 ─ 07/12/2023', '15/03/2024', '08/03/2024', '24/05/2024 ─ 25/05/2024', '26/01/2024', '01/12/2023', '05/04/2024', '22/03/2024', '12/04/2024', '16/02/2024', '24/05/2024', '11/11/2023', '19/04/2024', '02/02/2024', '31/05/2024', '01/12/2023', '10/11/2023', '21/12/2023 ─ 22/12/2023', '17/05/2024', '12/01/2024', '01/03/2024', '19/01/2024', '09/02/2024', '23/02/2024', '26/04/2024', '15/12/2023', '24/11/2023', '14/12/2023', '22/12/2023', '17/11/2023', '10/11/2023', '08/05/20

In [42]:
datos_4 = pd.DataFrame({'artista': artistas, 'fechas': fechas, 'eventos': num_eventos, 'precios': precios, 'enlaces': enlaces})
datos_4.head()


,artista,fechas,eventos,precios,enlaces
0,JÓVENES MÚSICOS III,05/04/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
1,Concierto Sinfónico de Abono A11 (Jueves),08/02/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
2,Concierto Sinfónico de Abono A17 (Jueves),11/04/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
3,Maximiliano Calvo - El Gallo,11/01/2024,1 Evento,"desde 16,50 €",https://www.entradas.com/artist/inverfest/maxi...
4,Ricardo Moya,21/01/2024,1 Evento,"desde 16,50 €",https://www.entradas.com/artist/inverfest/rica...


#### Para entrar en la siguiente página (Pág. 5)

In [43]:
driver.find_elements(By.XPATH, '//div/product-listing/listing-pagination/nav/ul/pagination-item[7]/a/i')[0].click()

In [44]:
table = driver.find_elements(By.XPATH, '//product-group-item')

In [45]:
elements = driver.find_elements(By.XPATH, '//div[@class="search-result-content"]')
elements

[<selenium.webdriver.remote.webelement.WebElement (session="fd0d5c95b8fc3784ead1aa0fe516404a", element="75E7392E3E82BE1DDA27F165946A2990_element_1224")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fd0d5c95b8fc3784ead1aa0fe516404a", element="75E7392E3E82BE1DDA27F165946A2990_element_1145")>]

In [46]:
data = []

for element in elements:
    artist_name = element.find_elements(By.XPATH, '//div[@class="event-listing-city theme-text-color"]')
    astists = [artist.text for artist in artist_name]
    date_elements = element.find_elements(By.XPATH, '//div//span[@class="listing-data theme-text-color"]//span')
    dates = [date.text for date in date_elements]
    price = element.find_elements(By.XPATH, '//div/listing-cta/event-status/div/span/span')
    precios = [precio.text for precio in price]
    enlace = element.find_elements(By.XPATH, '//div[2]/product-group-item/listing-item/article/div/listing-cta/a')
    enlaces = [enl.get_attribute('href') for enl in enlace]
    
    data.append([astists, dates, precios, enlaces])

for entry in data:
    print(entry)

[['Jaleos Jondos', 'Trovadores del Jazz', 'Schumann y Brahms - Ensemble Praeteritum', 'Aranlight Soul & Gospel - Jazz & the City', 'Mujeres Pop', 'Fred de Palma', 'Fran Perea "1 más 1 son 20"', 'Danza para Guitarra', 'Invisible Tattoo', 'Footloosing', 'Apartamentos Acapulco + Cora Yako', 'Luceral + Cherry Pie', 'La Serva Padrona - Aranjuez', 'Momentos Alhambra: Rikas', 'Aires de Persia', 'Diego Guerrero', 'Luna Ki', 'Kaze - Gira 2024', 'Rupatrupa', 'Mujer, inspiración y acción', 'The Music of Big Star + The Sadies', 'The Greatest Love Of All', 'Blues & Roots', 'Impulso presenta: Danilo Plessow (Motor City Drum Ensemble) + Javi Frias & Roberto Tempo', 'Gran Gala Flamenco - Madrid', 'Rolling Stones vs Beatles', 'Lo + actual', 'Homenaje a Pablo Milanés', 'Rawayana - ¿Quien trae las cornetas?', 'Rhythm Of The Dance', 'Eva Català Group', 'The Other Side - A Pink Floyd Live Experience', 'VibranDo', 'Lina', 'Marc Seguí - "AAAAAA Tour"', 'Giant Sand', 'Ters - #LaFdeF', 'Discotizer Presenta - C

In [47]:
artistas = entry[0]
fecha_eventos = entry[1]
precios = entry[2]

In [48]:
fechas = []
num_eventos = []

for i in range(0, len(fecha_eventos), 2):
    fechas.append(fecha_eventos[i])

for i in range(1, len(fecha_eventos), 2):
    num_eventos.append(fecha_eventos[i])

print(fechas)
print(num_eventos)

['05/11/2023 ─ 10/12/2023', '06/11/2023 ─ 18/12/2023', '16/11/2023', '15/12/2023', '10/11/2023', '26/03/2024 ─ 27/03/2024', '23/02/2024', '09/12/2023', '29/11/2023', '11/11/2023', '01/12/2023', '08/11/2023', '25/11/2023', '14/12/2023', '13/11/2023', '15/12/2023', '25/04/2024', '21/04/2024', '17/11/2023', '19/11/2023', '16/11/2023', '20/11/2023', '26/11/2023', '10/11/2023', '24/12/2023 ─ 01/01/2024', '17/11/2023', '01/12/2023', '24/03/2024', '07/04/2024 ─ 10/04/2024', '12/02/2024', '25/11/2023', '24/11/2023 ─ 17/12/2023', '17/12/2023', '12/03/2024', '20/04/2024', '26/11/2023', '16/02/2024', '18/11/2023', '23/12/2023', '08/12/2023', '18/11/2023', '18/11/2023', '07/11/2023', '25/04/2024 ─ 26/04/2024', '17/12/2023', '26/11/2023', '25/02/2024', '29/02/2024', '03/12/2023', '08/11/2023', '18/11/2023', '11/11/2023', '14/11/2023 ─ 15/11/2023', '01/12/2023 ─ 02/12/2023', '24/11/2023', '05/11/2023', '19/11/2023', '07/11/2023', '11/12/2023']
['38 eventos', '8 eventos', '1 Evento', '1 Evento', '1 E

In [49]:
datos_5 = pd.DataFrame({'artista': artistas, 'fechas': fechas, 'eventos': num_eventos, 'precios': precios, 'enlaces': enlaces})

In [50]:
eventos = pd.concat([datos_1, datos_2, datos_3, datos_4, datos_5])
eventos.tail()

,artista,fechas,eventos,precios,enlaces
54,La Movida Madrileña,24/11/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/conciertos-plu...
55,Momentos Alhambra: Sunny War,05/11/2023,1 Evento,Actualmente no disponible,https://www.entradas.com/artist/momentos-alham...
56,Noche de Música Española,19/11/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/noche-de-music...
57,Oneus,07/11/2023,1 Evento,"desde 55,00 €",https://www.entradas.com/artist/oneus/
58,15 Tangos de historia - La Porteña Tango - Madrid,11/12/2023,1 Evento,"desde 20,00 €",https://www.entradas.com/artist/la-portena-tan...


In [51]:
eventos.reset_index(inplace=True)

In [52]:
eventos.drop(columns="index",inplace=True)

In [53]:
eventos.head(20)

,artista,fechas,eventos,precios,enlaces
0,Cariño,13/01/2024,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/carino/carino-...
1,Iván Ferreiro,13/01/2024,1 Evento,"desde 38,50 €",https://www.entradas.com/artist/ivan-ferreiro/
2,The World Of Hans Zimmer,24/04/2024 ─ 29/04/2024,2 eventos,"desde 55,00 €",https://www.entradas.com/artist/hans-zimmer/
3,Triángulo de Amor Bizarro,29/12/2023,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/triangulo-de-a...
4,Nico B,14/01/2024,1 Evento,"desde 13,50 €",https://www.entradas.com/artist/nico-b/
5,Cala Vento,01/12/2023,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/cala-vento/
6,Sidonie,12/01/2024 ─ 14/01/2024,2 eventos,"desde 27,50 €",https://www.entradas.com/artist/sidonie/sidoni...
7,Belako - Sigo regando,20/01/2024,1 Evento,"desde 17,60 €",https://www.entradas.com/artist/belako/
8,Biznaga,26/01/2024,1 Evento,"desde 18,70 €",https://www.entradas.com/artist/biznaga/
9,Morreo - Alegría Club!,18/01/2024,1 Evento,"desde 14,50 €",https://www.entradas.com/artist/morreo/


In [54]:
eventos.tail(20)

,artista,fechas,eventos,precios,enlaces
199,Password - Sara Sánchez - Aranjuez,08/12/2023,1 Evento,"desde 16,00 €",https://www.entradas.com/artist/sara-sanchez/
200,José María Vitier,18/11/2023,1 Evento,"desde 12,00 €",https://www.entradas.com/artist/jose-maria-vit...
201,Fila 8: Clásicos del Cine Moderno - Jazz & the...,18/11/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/fila-8-clasico...
202,Momentos Alhambra: Laura Misch,07/11/2023,1 Evento,"desde 18,00 €",https://www.entradas.com/artist/momentos-alham...
203,Parov Stelar - Theater Tour,25/04/2024 ─ 26/04/2024,2 eventos,"desde 40,00 €",https://www.entradas.com/artist/parov-stelar/
204,Gala de Zarzuela en Navidad,17/12/2023,1 Evento,"desde 39,00 €",https://www.entradas.com/artist/gala-de-zarzue...
205,Zetazen,26/11/2023,1 Evento,"desde 22,00 €",https://www.entradas.com/artist/zetazen/
206,Paula Koops,25/02/2024,1 Evento,"desde 20,00 €",https://www.entradas.com/artist/paula-koops/
207,Vic Mirallas + invitados,29/02/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/vic-mirallas/
208,"Homenaje a Morricone, Zimmer, Williams, Hisaishi",03/12/2023,1 Evento,"desde 45,00 €",https://www.entradas.com/artist/homenaje-a-mor...


In [55]:
eventos = pd.DataFrame(eventos)

# Separa las fechas en fecha1 y fecha2

eventos['fecha1'] = eventos['fechas'].str.split(' ─ ').str[0]
eventos['fecha2'] = eventos['fechas'].str.split(' ─ ').str[1]

# Elimina espacios en blanco adicionales en las fechas

eventos['fecha1'] = eventos['fecha1'].str.strip()
eventos['fecha2'] = eventos['fecha2'].str.strip()

# Rellena las fechas faltantes en fecha2 con NaN

eventos['fecha2'].fillna('', inplace=True)

# Elimina la columna original 'fechas' si es necesario

eventos.drop('fechas', axis=1, inplace=True)

In [59]:
eventos

,artista,eventos,precios,enlaces,fecha1,fecha2
0,Cariño,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/carino/carino-...,13/01/2024,
1,Iván Ferreiro,1 Evento,"desde 38,50 €",https://www.entradas.com/artist/ivan-ferreiro/,13/01/2024,
2,The World Of Hans Zimmer,2 eventos,"desde 55,00 €",https://www.entradas.com/artist/hans-zimmer/,24/04/2024,29/04/2024
3,Triángulo de Amor Bizarro,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/triangulo-de-a...,29/12/2023,
4,Nico B,1 Evento,"desde 13,50 €",https://www.entradas.com/artist/nico-b/,14/01/2024,
...,...,...,...,...,...,...
214,Gala de Zarzuela en Navidad,1 Evento,"desde 39,00 €",https://www.entradas.com/artist/gala-de-zarzue...,17/12/2023,
215,Larkin Poe,1 Evento,"desde 25,00 €",https://www.entradas.com/artist/larkin-poe/,08/11/2023,
216,La Movida Madrileña,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/conciertos-plu...,24/11/2023,
217,Oneus,1 Evento,"desde 55,00 €",https://www.entradas.com/artist/oneus/,07/11/2023,


- Quiero duplicar las filas en el caso de que las fechas sean múltiples, ya que me servirá para la creacion de mi base de datos posterior y para su estudio comparado. 

In [56]:
len(eventos)

219

In [57]:
nuevas_filas = []

# Iterar a través de las filas
for _, fila in eventos.iterrows():
    nuevas_filas.append(fila)
    if fila['fecha2']:
        nueva_fila = fila.copy()
        nueva_fila['fecha1'] = fila['fecha2']
        nueva_fila['fecha2'] = ''
        nuevas_filas.append(nueva_fila)

# Crear un nuevo DataFrame con las filas duplicadas
df_resultante = pd.DataFrame(nuevas_filas, columns = eventos.columns)

# Imprimir el DataFrame resultante
print(df_resultante)

                                               artista    eventos  \
0                                               Cariño   1 Evento   
1                                        Iván Ferreiro   1 Evento   
2                             The World Of Hans Zimmer  2 eventos   
2                             The World Of Hans Zimmer  2 eventos   
3                            Triángulo de Amor Bizarro   1 Evento   
..                                                 ...        ...   
214                                La Movida Madrileña   1 Evento   
215                       Momentos Alhambra: Sunny War   1 Evento   
216                           Noche de Música Española   1 Evento   
217                                              Oneus   1 Evento   
218  15 Tangos de historia - La Porteña Tango - Madrid   1 Evento   

                       precios  \
0                desde 19,80 €   
1                desde 38,50 €   
2                desde 55,00 €   
2                desde 55,00 €   
3

In [58]:
df_resultante.head()

,artista,eventos,precios,enlaces,fecha1,fecha2
0,Cariño,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/carino/carino-...,13/01/2024,
1,Iván Ferreiro,1 Evento,"desde 38,50 €",https://www.entradas.com/artist/ivan-ferreiro/,13/01/2024,
2,The World Of Hans Zimmer,2 eventos,"desde 55,00 €",https://www.entradas.com/artist/hans-zimmer/,24/04/2024,29/04/2024
2,The World Of Hans Zimmer,2 eventos,"desde 55,00 €",https://www.entradas.com/artist/hans-zimmer/,29/04/2024,
3,Triángulo de Amor Bizarro,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/triangulo-de-a...,29/12/2023,


In [59]:
len(df_resultante)

251

- le cambio el nombre, y así hago una copia del dataframe

In [60]:
df_event = df_resultante.copy()

In [62]:
df_event = df_event.drop('fecha2', axis=1)


In [63]:
df_event.head()

,artista,eventos,precios,enlaces,fecha1
0,Cariño,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/carino/carino-...,13/01/2024
1,Iván Ferreiro,1 Evento,"desde 38,50 €",https://www.entradas.com/artist/ivan-ferreiro/,13/01/2024
2,The World Of Hans Zimmer,2 eventos,"desde 55,00 €",https://www.entradas.com/artist/hans-zimmer/,24/04/2024
2,The World Of Hans Zimmer,2 eventos,"desde 55,00 €",https://www.entradas.com/artist/hans-zimmer/,29/04/2024
3,Triángulo de Amor Bizarro,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/triangulo-de-a...,29/12/2023


In [64]:
df_event = df_event.rename(columns={'fecha1': 'fechas'})

In [65]:
df_event = df_event[['artista', 'eventos', 'fechas', 'precios', 'enlaces']]

In [66]:
df_event.head()

,artista,eventos,fechas,precios,enlaces
0,Cariño,1 Evento,13/01/2024,"desde 19,80 €",https://www.entradas.com/artist/carino/carino-...
1,Iván Ferreiro,1 Evento,13/01/2024,"desde 38,50 €",https://www.entradas.com/artist/ivan-ferreiro/
2,The World Of Hans Zimmer,2 eventos,24/04/2024,"desde 55,00 €",https://www.entradas.com/artist/hans-zimmer/
2,The World Of Hans Zimmer,2 eventos,29/04/2024,"desde 55,00 €",https://www.entradas.com/artist/hans-zimmer/
3,Triángulo de Amor Bizarro,1 Evento,29/12/2023,"desde 19,80 €",https://www.entradas.com/artist/triangulo-de-a...


In [68]:

df_event['fechas'] = pd.to_datetime(df_event['fechas'], format='%d/%m/%Y', errors='coerce')

In [69]:

# Separar las fechas en columnas de día, mes y año

df_event['dia'] = df_event['fechas'].dt.day
df_event['mes'] = df_event['fechas'].dt.month
df_event['año'] = df_event['fechas'].dt.year


In [70]:
df_event.head()

,artista,eventos,fechas,precios,enlaces,dia,mes,año
0,Cariño,1 Evento,2024-01-13,"desde 19,80 €",https://www.entradas.com/artist/carino/carino-...,13,1,2024
1,Iván Ferreiro,1 Evento,2024-01-13,"desde 38,50 €",https://www.entradas.com/artist/ivan-ferreiro/,13,1,2024
2,The World Of Hans Zimmer,2 eventos,2024-04-24,"desde 55,00 €",https://www.entradas.com/artist/hans-zimmer/,24,4,2024
2,The World Of Hans Zimmer,2 eventos,2024-04-29,"desde 55,00 €",https://www.entradas.com/artist/hans-zimmer/,29,4,2024
3,Triángulo de Amor Bizarro,1 Evento,2023-12-29,"desde 19,80 €",https://www.entradas.com/artist/triangulo-de-a...,29,12,2023


In [71]:
df_event.drop('fechas', axis=1, inplace=True)

In [72]:
column_order = ['artista', 'eventos', 'dia', 'mes', 'año', 'precios', 'enlaces']
df_event = df_event[column_order]

In [73]:
df_event.head()

,artista,eventos,dia,mes,año,precios,enlaces
0,Cariño,1 Evento,13,1,2024,"desde 19,80 €",https://www.entradas.com/artist/carino/carino-...
1,Iván Ferreiro,1 Evento,13,1,2024,"desde 38,50 €",https://www.entradas.com/artist/ivan-ferreiro/
2,The World Of Hans Zimmer,2 eventos,24,4,2024,"desde 55,00 €",https://www.entradas.com/artist/hans-zimmer/
2,The World Of Hans Zimmer,2 eventos,29,4,2024,"desde 55,00 €",https://www.entradas.com/artist/hans-zimmer/
3,Triángulo de Amor Bizarro,1 Evento,29,12,2023,"desde 19,80 €",https://www.entradas.com/artist/triangulo-de-a...


- Una vez que he terminado de trabajar con la web, salgo de esta:

In [74]:
driver.quit()

- Una vez he importado el dataframe como .csv lo marco para que no se me vuelva a convertir.

In [76]:
#df_event.to_csv('eventos.csv', index=False)